In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

from skimage import io, filters, measure

import warnings
warnings.filterwarnings('ignore')

In [ ]:
# define input path
inputpath = '' # example '/Users/projects/'

# read reference image
ref_img = io.imread(os.path.join(inputpath, '20460260_Scan1_Core[1,2,A]_[12663,40371]_component_data.tif - resolution #3.tif'))
nuclei = ref_img[0,:,:]

In [ ]:
from stardist.models import StarDist2D
from stardist.plot import render_label
from csbdeep.utils import normalize

# prints a list of available models
StarDist2D.from_pretrained()

# creates a pretrained model
model = StarDist2D.from_pretrained('2D_versatile_fluo')

# get predictions for nuclei 
labels, _ = model.predict_instances(normalize(nuclei))

# get binary mask
binary_mask = labels.copy()
binary_mask[binary_mask > 0] = 1

io.imsave(inputpath + 'labels.tif',labels)
io.imsave(inputpath + 'binary.tif',binary_mask)

In [ ]:
# display the output
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 10))
ax1.imshow(nuclei, cmap="gray", vmin=0, vmax=np.amax(nuclei))
ax1.axis("off")
ax1.set_title('input image')

ax2.imshow(render_label(labels, img=nuclei))
ax2.axis("off")
ax2.set_title('prediction + input overlay')


In [ ]:
from skimage.filters import threshold_otsu

# save measures for all channels in a dictionary
#measures = {} 

# for each channel excluding nuclei channel
for x in range(ref_img.shape[0]-1):
    print("\n" +"-"*20)
    print('channel: ', x+2)
    
    # select channel to be analyzed
    ref_channel = ref_img[(x+1),:,:]
    
    # get threshold value
    threshold = filters.threshold_otsu(ref_channel) # get threshold value
    print('threshold: ', threshold)
    
    if x == 0: # then, get shape measurements
        props_shape = measure.regionprops_table(labels, ref_channel, properties=['label','area','axis_major_length',
                                                                       'axis_minor_length','feret_diameter_max'])
        
    props_intens = measure.regionprops_table(labels, ref_channel, properties=['intensity_max','intensity_min',
                                                                              'intensity_mean'])
    if x == 0:
        props = pd.concat([pd.DataFrame(props_shape), pd.DataFrame(props_intens)], axis=1)
    else:
        props = pd.concat([pd.DataFrame(props), pd.DataFrame(props_intens)], axis=1)
        
props.to_csv(os.path.join(inputpath, 'measures.csv'))
    